In [1]:
import cv2
import numpy as np
import pyrealsense2
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib as mpl
import pyrealsense2 as rs
import copy
import math
from scipy.spatial import distance

In [2]:
class DepthCamera:
    def __init__(self):
        # Configure depth and color streams
        self.pipeline = rs.pipeline()
        config = rs.config()

        # Get device product line for setting a supporting resolution
        pipeline_wrapper = rs.pipeline_wrapper(self.pipeline)
        pipeline_profile = config.resolve(pipeline_wrapper)
        device = pipeline_profile.get_device()
        device_product_line = str(device.get_info(rs.camera_info.product_line))

        config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
        config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
        # Start streaming
        self.pipeline.start(config)

  
    def get_frame(self):
        frames = self.pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        if not depth_frame or not color_frame:
            return False, None, None
        return True, depth_image, color_image

    def video(self):
        align_to = rs.stream.color
        align = rs.align(align_to)
        for i in range(10):
            self.pipeline.wait_for_frames()
        while True:
            frames = self.pipeline.wait_for_frames()
            aligned_frames = align.process(frames)
            color_frame = aligned_frames.get_color_frame()
            depth_frame = aligned_frames.get_depth_frame()

            depth_frame = depth_frame

            color_image = np.asanyarray(color_frame.get_data())
            color_intrin = color_frame.profile.as_video_stream_profile().intrinsics

            depth_color_frame = rs.colorizer().colorize(depth_frame)

            # Convert depth_frame to numpy array to render image in opencv
            depth_color_image = np.asanyarray(depth_color_frame.get_data())
            color_image = np.asanyarray(color_frame.get_data())
            color_cvt = cv2.cvtColor(color_image, cv2.COLOR_BGR2RGB)
            

            break
            # Render image in opencv window
            # Create opencv window to render image in
        return color_intrin


    def release(self):
        self.pipeline.stop()

In [3]:
# Constants.
INPUT_WIDTH = 640
INPUT_HEIGHT = 640
SCORE_THRESHOLD = 0.5
NMS_THRESHOLD = 0.5
CONFIDENCE_THRESHOLD = 0.5

# Text parameters.
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.7
THICKNESS = 1

# Colors
BLACK  = (0,0,0)
BLUE   = (255,178,50)
YELLOW = (0,255,255)
RED = (0,0,255)


In [4]:
def draw_label(input_image, label, left, top):
    """Draw text onto image at location."""
    
    # Get text size.
    text_size = cv2.getTextSize(label, FONT_FACE, FONT_SCALE, THICKNESS)
    dim, baseline = text_size[0], text_size[1]
    # Use text size to create a BLACK rectangle. 
    cv2.rectangle(input_image, (left, top), (left + dim[0], top + dim[1] + baseline), BLACK, cv2.FILLED);
    # Display text inside the rectangle.
    cv2.putText(input_image, label, (left, top + dim[1]), FONT_FACE, FONT_SCALE, YELLOW, THICKNESS, cv2.LINE_AA)


In [5]:

def pre_process(input_image, net):
    # Create a 4D blob from a frame.
    blob = cv2.dnn.blobFromImage(input_image, 1/255, (INPUT_WIDTH, INPUT_HEIGHT), [0,0,0], 1, crop=False)

    # Sets the input to the network.
    net.setInput(blob)

    # Runs the forward pass to get output of the output layers.
    output_layers = net.getUnconnectedOutLayersNames()
    outputs = net.forward(output_layers)
    # print(outputs[0].shape)

    return outputs


In [6]:

def post_process(input_image, outputs):
    # Lists to hold respective values while unwrapping.
    class_ids = []
    confidences = []
    boxes = []

    # Rows.
    rows = outputs[0].shape[1]

    image_height, image_width = input_image.shape[:2]

    # Resizing factor.
    x_factor = image_width / INPUT_WIDTH
    y_factor =  image_height / INPUT_HEIGHT

    # Iterate through 25200 detections.
    for r in range(rows):
        row = outputs[0][0][r]
        confidence = row[4]

        # Discard bad detections and continue.
        if confidence >= CONFIDENCE_THRESHOLD:
            classes_scores = row[5:]

            # Get the index of max class score.
            class_id = np.argmax(classes_scores)

            #  Continue if the class score is above threshold.
            if (classes_scores[class_id] > SCORE_THRESHOLD):
                confidences.append(confidence)
                class_ids.append(class_id)

                cx, cy, w, h = row[0], row[1], row[2], row[3]
                x = int((cx - w/2) * x_factor)
                y = int((cy - h/2) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                
                point1= ((int(x),int(y+h)))
                point2= ((int(x),int(y)))
                point3= ((int(x+w),int(y+h)))
                distance1= depth_frame[point1[1], point1[0]]
                distance2= depth_frame[point2[1], point2[0]]
                distance3= depth_frame[point3[1], point3[0]]
                
                d1 = rs.rs2_deproject_pixel_to_point(color_intrin, [x, y+h], distance1/1000)
                d2 = rs.rs2_deproject_pixel_to_point(color_intrin, [x, y], distance2/1000)
                d3 = rs.rs2_deproject_pixel_to_point(color_intrin, [x+w, y+h], distance3/1000)
                
                dist = int(math.sqrt(math.pow(d1[0] - d2[0], 2) + math.pow(d1[1] - d2[1],2) + math.pow( d1[2] - d2[2], 2))*100)
                dist2 = int(math.sqrt(math.pow(d3[0] - d1[0], 2) + math.pow(d3[1] - d1[1],2) + math.pow( d3[2] - d1[2], 2))*100)
    
                cv2.putText(input_image, "{}cm".format(dist), (point1[0], point1[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
                cv2.putText(input_image, "{}cm".format(dist2), (point3[0], point3[1] - 20), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 2)
                
                
 
                box = np.array([x, y, width, height])
                boxes.append(box)

    # Perform non maximum suppression to eliminate redundant overlapping boxes with
    # lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    for i in indices:
        box = boxes[i]
        x = box[0]
        y = box[1]
        width = box[2]
        height = box[3]
        cv2.rectangle(input_image, (x, y), (x + width, y + height), BLUE, 3*THICKNESS)
        label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])
        draw_label(input_image, label, x, y)

    return input_image

In [7]:
if __name__ == '__main__':
    # Load class names.
    classesFile = "coco.names"
    classes = None
    with open(classesFile, 'rt') as f:
        classes = f.read().rstrip('\n').split('\n')

    # Load image.
    dc = DepthCamera()
    ret, depth_frame, image = dc.get_frame()
    frame = image
    color_intrin = dc.video()
    # Give the weight files to the model and load the network using them.
    modelWeights = "best.onnx"
    net = cv2.dnn.readNet(modelWeights)

    # Process image.-
    detections = pre_process(frame, net)
    img = post_process(frame.copy(), detections)

    # Put efficiency information. The function getPerfProfile returns the overall time for inference(t) and the timings for each of the layers(in layersTimes)
    t, _ = net.getPerfProfile()
    label = 'Inference time: %.2f ms' % (t * 1000.0 / cv2.getTickFrequency())
    print(label)
    cv2.putText(img, label, (20, 40), FONT_FACE, FONT_SCALE, RED, THICKNESS, cv2.LINE_AA)

    cv2.imshow('Output', img)
    cv2.waitKey(0)

Inference time: 211.78 ms
